In [1]:
# для запросов
import requests

# для работы с JSON
import json

# для работы с папками
import os

from time import sleep

import math

# 
from datetime import timedelta, date

# прогресс-бар
from tqdm.notebook import trange, tqdm

# для дафреймов
import pandas as pd

In [15]:
# путь к файлам

file_path = 'C:/00_Data/temp_data/'

token_list_path = 'C:/work/keys/vk.txt' # путь к файлу с токеном

In [16]:
# загружаю токен

with open(token_list_path, 'r') as f:
    for line in f:
        vk_key = line

In [59]:
# настройки для парсинга
version = 5.131


In [4]:
post_number = 10
offset = 0
all_post = []

In [6]:
# количество постов на стене

all_post_number = requests.get('https://api.vk.com/method/wall.get',
                               params={
                                   'access_token': vk_key,
                                   'v':version,
                                   'domain': domain,
                               }).json()['response']['count']

In [7]:
# количество страниц

page_number = math.ceil(all_post_number / 100)
page_number

97

In [ ]:
# код парсера

data = []

for page in tqdm(range(0, page_number)):
    
    
    response = requests.get('https://api.vk.com/method/wall.get',
                            params={
                                'access_token': vk_key,
                                'v':version,
                                'domain': domain,
                                'count': post_number,
                                'offset': offset,
                                'extended': 1
                            })
   
    
    sleep(0.5)
    offset += 100
    
    for item in response.json()['response']['items']:
        video = 0
        photo = 0
        
        owner_id = item['owner_id']
        autor_id = item['from_id']
        post_date = item['date']
        post_type = item['post_type']
        marked_as_ads = item['marked_as_ads']
        post_text = item['text']
        
        try:
            attachment_number = len(item['attachments'])
            
            for i in item['attachments']:
                if i['type'] == 'video':
                    video += 1
                
                if i['type'] == 'photo':
                    photo += 1
            
            comment_number = item['comments']['count']
            like_number = item['likes']['count']
        
        except:
            attachment_number = 0
        
        
        data.append([owner_id, autor_id,
                     post_date, post_type, marked_as_ads,
                     post_text,
                     attachment_number, video, photo,
                     comment_number, like_number])

In [ ]:
# преобразовываю в датафрейм

head = ['owner_id', 'autor_id',
        'post_date', 'post_type', 'marked_as_ads',
        'post_text',
        'attachment_number', 'video', 'photo',
        'comment_number', 'like_number']

work_wall = pd.DataFrame(data, columns=head)

In [ ]:
work_wall.sample(3)

In [ ]:
work_wall.shape

In [ ]:
all_post_number

In [ ]:
work_wall.to_excel(file_path + 'VK_' + domain +'.xlsx', index=False)

### Вытаскиваю id групп по нужному запросу

In [102]:
# запрос
response = requests.get('https://api.vk.com/method/groups.search',
                        params={
                            'access_token': vk_key,
                            'v': version,
                            'type': 'group',
                            'country_id': 1,
                            'count': 1000,
                            'sort': 6,
                            'q': 'кладоискатели'
                            }).json()['response']['items']

In [103]:
group_id_list = []

for i in response:
    group_id_list.append(i['id'])

### Данные групп

Название, количество участников, город, описание, направление

In [116]:
data_group = []

# настройки для парсинга
count = 1000
offset = 0

for group_id in tqdm(group_id_list):
    
    # запрос
    response = requests.get('https://api.vk.com/method/groups.getById',
                            params={
                                'access_token': vk_key,
                                'v':version,
                                'group_id': group_id,
                                'fields': 'members_count,city,description,activity'
                            }).json()['response'][0]
    
    sleep(0.5)
    
    
    name = response['name']
    screen_name = response['screen_name']
    members_count = response['members_count']
    
    try:
        city_id = response['city']['id']
        city_title = response['city']['title']
        
    except:
        city_id = 'none'
        city_title = 'none'
            
    try:
        group_description = response['description']
        
    except:
        group_description = 'none'
    
    activity = response['activity']
    
    is_closed = response['is_closed']
    
    data_group.append([group_id, name, screen_name, members_count, city_id, city_title, group_description, activity, is_closed])

  0%|          | 0/732 [00:00<?, ?it/s]

In [117]:
head = ['group_id', 'name', 'screen_name', 'members_count', 'city_id', 'city_title', 'group_description', 'activity', 'closed']

work_groups = pd.DataFrame(data_group, columns=head)

In [118]:
work_groups.shape

(732, 9)

In [119]:
work_groups.head(5)

,group_id,name,screen_name,members_count,city_id,city_title,group_description,activity,closed
0,30541427,Кладоискатель Металлоискатель Нумизмат,kladoiskatell,108565,1,Москва,"Хобби - частная, честная АРХЕОЛОГИЯ\nГруппа не...",Открытая группа,0
1,57592667,Minelab detectors - сообщество кладоискателей.,minelabdetectors,88492,1,Москва,Во все времена люди искали и находили клады и...,Открытая группа,0
2,108651950,КЛАДОИСКАТЕЛЬ..СТАРИННЫЕ КАРТЫ.,magazinkart,81103,1,Москва,Составление старинных карт для Кладоискателей ...,Открытая группа,0
3,47055113,Кладоискатели Нумизматы,poiskfan,67541,none,none,"vk.com/poiskfan - это группа Кладоискателей, Н...",Открытая группа,0
4,46232640,У Деда Мити | Металлоискатели | Кладоискатели,udedamiti,60764,270,Подольск,Официальное сообщество магазина для поисковико...,Открытая группа,0


In [120]:
work_groups[work_groups.closed == 1].shape

(59, 9)

In [127]:
work_groups_need = work_groups[
    (work_groups.members_count >= 1000) &
    (work_groups.closed == 0)]

In [128]:
# сбрасываю индекс

work_groups_need.reset_index(inplace = True, drop = True)

In [129]:
work_groups_need.shape

(126, 9)

In [130]:
work_groups_need.sample(2)

,group_id,name,screen_name,members_count,city_id,city_title,group_description,activity,closed
83,51960292,Коллекционер-кладоискатель,xobbbi,1970,18022,Иланский,"Предметы коллекционирования. Покупка, продажа,...",Открытая группа,0
100,97128072,АйДаКлад.Кладоискатели.Металлоискатели.Нумизмат,idaklad,1561,1,Москва,Магазин товаров для поиска реликвий «АйДаклад»...,Открытая группа,0


In [125]:
#work_groups.to_excel(file_path + 'groups_info.xlsx', index=False)

In [126]:
#work_groups.to_csv(file_path + 'groups_info.csv', index=False)

### Вытаскиваю пользователей

In [133]:
# парсинг подписчиков по группам

data_followers = [] # список для данных
count = 1000 # количество юзверей, которое я беру за 1 раз

for i in tqdm(range(0, work_groups_need.shape[0])):
    
    offset = 0
    
    group_id = work_groups_need.group_id[i]
    screen_name = work_groups_need.screen_name[i]
        
            
    # количество юзеров
    user_number = math.ceil(work_groups_need.members_count[i] / 1000)
    
    try: # проверяю скрыты ли пользователи
        error_msg = requests.get('https://api.vk.com/method/groups.getMembers',
                                 params={
                                     'access_token': vk_key,
                                     'v': version,
                                     'group_id': group_id,
                                     'count': count,
                                     'offset': 0,
                                     'fields': 'city, country, bdate, sex, education, domain'
                                 }).json()['error']['error_msg']
    except:
        error_msg = 'none'
        
        if error_msg == 'none': # если пользователи открыты, собираю
            
            # код парсера
            for page in range(0, user_number):
                
                response = requests.get('https://api.vk.com/method/groups.getMembers',
                                        params={
                                            'access_token': vk_key,
                                            'v': version,
                                            'group_id': group_id,
                                            'count': count,
                                            'offset': offset,
                                            'fields': 'city, country, bdate, sex, education, domain'
                                        })
                
                sleep(0.5)
                offset += 1000
                
                # разбираю ответ сервера
                for item in response.json()['response']['items']:
                    
                    user_id = item['id']
                    first_name = item['first_name']
                    last_name = item['last_name']
                    sex = item['sex']

                    try:
                        bdate = item['bdate']

                    except:
                        bdate = 'none'

                    try:
                        city_title = item['city']['title']
                        city_id = item['city']['id']

                    except:
                        city_title = 'none'
                        city_id = 'none'

                    try:
                        country_title = item['country']['title']
                        country_id = item['country']['id']

                    except:
                        country_title = 'none'
                        country_id = 'none'

                    try:
                        university_id = item['university']
                        university_name = ['university_name']

                    except:
                        university_id = 'none'
                        university_name = 'none'

                    try:
                        faculty_name = item['university']['faculty_name']

                    except:
                        faculty_name = 'none'

                    try:
                        education_status = item['education_status']

                    except:
                        education_status = 'none'

                    try:
                        account_type = item['is_closed']

                    except:
                        account_type = 'none'


                    data_followers.append([user_id, first_name, last_name, sex, bdate,
                                 city_title, city_id, country_title, country_id,
                                 education_status,
                                 university_id, university_name, faculty_name,                     
                                 account_type, group_id, screen_name])

  0%|          | 0/126 [00:00<?, ?it/s]

In [134]:
head = ['user_id', 'first_name', 'last_name', 'sex', 'bdate',
        'city_title', 'city_id', 'country_title', 'country_id',
        'education_status', 'university_id', 'university_name', 'faculty_name',
        'account_type', 'group_id', 'screen_name']

work_followers = pd.DataFrame(data_followers, columns=head)

In [135]:
# удаляю криво загруженные и ненужные столбцы

work_followers.drop(columns=['university_id', 'university_name', 'faculty_name'], inplace=True)

In [136]:
work_followers.head()

,user_id,first_name,last_name,sex,bdate,city_title,city_id,country_title,country_id,education_status,account_type,group_id,screen_name
0,410,Alexander,Obl,1,19.6,none,none,none,none,none,False,30541427,kladoiskatell
1,2994,Ангелина,Иванова,1,9.11.2001,none,none,none,none,none,False,30541427,kladoiskatell
2,4643,Катерина,Еремеева,1,24.11.1988,Москва,1,Россия,1,Выпускница (специалист),False,30541427,kladoiskatell
3,4668,Михаил,Зверев,2,10.7.1985,Пенза,109,Россия,1,none,False,30541427,kladoiskatell
4,5438,Миша,Самохин,2,none,Санкт-Петербург,2,Россия,1,none,False,30541427,kladoiskatell


In [137]:
work_followers.shape

(939738, 13)

In [152]:
#work_followers.to_csv(file_path + 'all_users_vk.csv', index=False)

In [138]:
# количество уникальных юзеров

work_followers.user_id.nunique()

597450

In [183]:
users_unique = (work_followers[
    (work_followers.country_id == 1) &
    (work_followers.city_id != 'none')].user_id
                .value_counts().reset_index()
                .rename(columns={'user_id': 'group_number', 'index': 'user_id'}))

In [184]:
users_unique.shape

(303507, 2)

### Собираю данные юзеров

In [269]:
# data_users = []

for user_id in tqdm(users_unique.user_id[149742:150001]):
    response = requests.get('https://api.vk.com/method/users.get',
                        params={
                            'access_token': vk_key,
                            'v':version,
                            'user_ids': user_id,
                            'fields': 'counters'                           
                            }).json()['response'][0]
    sleep(0.3)
    
    # разбираю ответ сервера
    
    try:
        deactivated = response['deactivated']
        
    except:
        deactivated = 'none'
        
    try:
        profile_closed = response['is_closed']
        
    except:
        profile_closed = 'none'
        
        
    try:
        friend_number = response['counters']['friends']
        
    except:
        friend_number = 'none'
        
        
    try:
        group_number = response['counters']['groups']
        
    except:
        group_number = 'none'
        
        
    data_users.append([user_id, deactivated, profile_closed, friend_number, group_number])

  0%|          | 0/259 [00:00<?, ?it/s]

In [270]:
head = ['user_id', 'deactivated', 'profile_closed', 'friend_number', 'group_number']

work_users = pd.DataFrame(data_users, columns=head)

In [276]:
work_users.tail(5)

,user_id,deactivated,profile_closed,friend_number,group_number
149997,222823304,none,False,245,none
149998,376072863,none,False,122,25
149999,171147938,none,True,166,none
150000,78328017,none,False,285,none
150001,553410859,none,False,31,1069


In [272]:
#work_users.to_csv(file_path + 'users_vk1.csv', index=False)

In [275]:
work_users.shape

(150002, 5)

In [284]:
work_followers.head(2)

,user_id,first_name,last_name,sex,bdate,city_title,city_id,country_title,country_id,education_status,account_type,group_id,screen_name
0,410,Alexander,Obl,1,19.6,none,none,none,none,none,False,30541427,kladoiskatell
1,2994,Ангелина,Иванова,1,9.11.2001,none,none,none,none,none,False,30541427,kladoiskatell


###  Определяю регион

In [307]:
city_list = list(work_followers[work_followers.city_title != 'none'].city_title.unique())

In [308]:
len(city_list)

11409

In [373]:
vk_key = 'e10dbcb8e0a4b1c0ba7eb2d31a2d4311c3596827e73c59675e1a875028a8fcd6d3d29d40975d7ec501667'

In [374]:
# data_city = []

for city in tqdm(city_list[10399:]):
    
    response = requests.get('https://api.vk.com/method/database.getCities',
                        params={
                            'access_token': vk_key,
                            'v':version,
                            'country_id': 1,
                            'count': 1000,
                            'q': city
                        }).json()['response']['items']
    sleep(0.3)
    
    for city in response:
        
        city_id = city['id']
        city_title = city['title']
        
        try:
            area = city['area']
            
        except:
            area = 'none'
            
        try:
            region = city['region']
            
        except:
            region = 'none'
        
        
        data_city.append([city_id, city_title, area, region])

  0%|          | 0/1010 [00:00<?, ?it/s]

In [375]:
head = ['city_id', 'city_title', 'area', 'region']

city_database = pd.DataFrame(data_city, columns=head)

In [376]:
city_database.tail(5)

,city_id,city_title,area,region
70275,1046131,Кузьмолово,Всеволожский район,Ленинградская область
70276,1031136,Железнодорожное,Краснознаменский район,Калининградская область
70277,1500029,Железнодорожное,Бахчисарайский,Крым
70278,5479204,Тельма (Железнодорожное МО),Усольский район,Иркутская область
70279,1023711,Починок (Железнодорожное с/п),Шекснинский район,Вологодская область


In [377]:
city_database.shape

(70280, 4)

In [378]:
city_database.drop_duplicates(inplace=True)

In [379]:
city_database.shape

(54547, 4)

In [380]:
city_database.to_csv(file_path + 'city_database.csv', index=False)